## Libs

In [1]:
import requests

In [2]:
import numpy as np
import pandas as pd

In [14]:
from tqdm.notebook import tqdm, trange

## Inits

In [8]:
baseURL = "https://ikzttp.mypinata.cloud/ipfs/QmeBWSnYPEnUimvpPfNHuvgcK9wFH9Sa6cZ4KDfgkfJJis/"

## Data

### Test Ping

In [19]:
r = requests.get(baseURL + "1").json()
r

{'name': 'Azuki #1',
 'image': 'https://ikzttp.mypinata.cloud/ipfs/QmYDvPAXtiJg7s8JdRBSLWdgSphQdac8j1YuQNNxcGE1hg/1.png',
 'attributes': [{'trait_type': 'Type', 'value': 'Human'},
  {'trait_type': 'Hair', 'value': 'Pink Hairband'},
  {'trait_type': 'Clothing', 'value': 'White Qipao with Fur'},
  {'trait_type': 'Eyes', 'value': 'Daydreaming'},
  {'trait_type': 'Mouth', 'value': 'Lipstick'},
  {'trait_type': 'Offhand', 'value': 'Gloves'},
  {'trait_type': 'Background', 'value': 'Off White D'}]}

### Scrape

In [74]:
df = pd.DataFrame()
for i in trange(9999):
    r = requests.get(baseURL + str(i))
    if (r.status_code==200):
        df = df.append(pd.DataFrame([r.json()]))

  0%|          | 0/9999 [00:00<?, ?it/s]

In [75]:
# pickle
df.to_pickle("./data/raw/azuki_raw.pkl")

### Transform

,name,image,attributes,id
0,Azuki #0,https://ikzttp.mypinata.cloud/ipfs/QmYDvPAXtiJ...,"[{'trait_type': 'Type', 'value': 'Human'}, {'t...",0
1,Azuki #1,https://ikzttp.mypinata.cloud/ipfs/QmYDvPAXtiJ...,"[{'trait_type': 'Type', 'value': 'Human'}, {'t...",1
2,Azuki #2,https://ikzttp.mypinata.cloud/ipfs/QmYDvPAXtiJ...,"[{'trait_type': 'Type', 'value': 'Human'}, {'t...",2
3,Azuki #3,https://ikzttp.mypinata.cloud/ipfs/QmYDvPAXtiJ...,"[{'trait_type': 'Type', 'value': 'Human'}, {'t...",3
4,Azuki #4,https://ikzttp.mypinata.cloud/ipfs/QmYDvPAXtiJ...,"[{'trait_type': 'Type', 'value': 'Human'}, {'t...",4


In [69]:
df.reset_index(drop=True,inplace=True)

In [126]:
def trait(x, trait_type):
    for r in x:
        if r["trait_type"]==trait_type:
            return r["value"]            

In [43]:
# convert attributes column to key-value
df["traits"] = df["attributes"].apply(lambda x: [{r["trait_type"]:r["value"]} for r in x])

In [133]:
# so not all entries have all the traits so will be doing a parser for each trait type

traits = ["Type", "Face", "Hair", "Headgear", "Eyes", "Ear", 
         "Mouth", "Neck", "Clothing", "Offhand", "Background", "Special"]

for t in traits:
    df[t]= df["attributes"].apply(lambda x: trait(x, t))

In [136]:
df.info() # some nulls

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9999 entries, 0 to 9998
Data columns (total 16 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   name        9999 non-null   object
 1   image       9999 non-null   object
 2   attributes  9999 non-null   object
 3   id          9999 non-null   object
 4   Type        9999 non-null   object
 5   Face        3209 non-null   object
 6   Hair        9991 non-null   object
 7   Headgear    3487 non-null   object
 8   Eyes        9999 non-null   object
 9   Ear         1819 non-null   object
 10  Mouth       9999 non-null   object
 11  Neck        2254 non-null   object
 12  Clothing    9994 non-null   object
 13  Offhand     6834 non-null   object
 14  Background  9999 non-null   object
 15  Special     629 non-null    object
dtypes: object(16)
memory usage: 1.2+ MB


In [139]:
# get a required columns and save to csv
df[["id", "name", "image"] + traits].to_csv("./data/final/azuki.csv", index=False)

In [140]:
# save a pickle
df[["id", "name", "image", "attributes"] + traits].to_pickle("./data/interim/azuki.pkl")